# Syft Keras सह खाजगी भविष्यवाण्या

## चरण 3: Syft Keras वापरुन खाजगी भविष्यवाणी - सर्व्हिंग (क्लायंट)

अनुवादक/संपादक:
- Krunal Kshirsagar - Twitter: [@krunal_wrote](https://twitter.com/krunal_wrote) - Github: [@Noob-can-Compile](https://github.com/Noob-can-Compile)

अभिनंदन! आपल्या मॉडेलला सामान्य Keras प्रशिक्षण दिल्यानंतर आणि त्यास Syft Keras सह सुरक्षित केल्यानंतर आपण काही खाजगी भविष्यवाण्यांसाठी विनंती करण्यास तयार आहात.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import syft as sy

## डेटा

येथे, आपण आपला MNIST डेटा तयार करतो. हे प्रशिक्षणादरम्यान आपण कसे पूर्वप्रक्रिया केले ते सारखेच आहे.

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## मॉडेलला कनेक्ट करा

मॉडेलची चौकशी करण्यापूर्वी, आपल्याला त्यास फक्त कनेक्ट करावे लागेल. असे करण्यासाठी, आपण एक क्लायंट तयार करू शकता. मग तंतोतंत समान तीन TFEWorkers (`alice` , `bob` , और `carol`) आणि क्लस्टर परिभाषित करा. शेवटी `connect_to_model` कॉल करा. हे क्लायंट बाजूस एक TFE क्यूइंग सर्व्हर तयार करते जो **भाग 13b** मध्ये `model.serve ()` द्वारे सेट अप केलेल्या रांगेत असलेल्या सर्व्हरशी जोडला जातो. पूर्वानुमान विनंतीमध्ये शेअर्स सबमिट करण्यापूर्वी प्लेन टेक्स्ट डेटा गुप्तपणे सामायिक करण्यासाठी रांग जबाबदार असेल.

In [3]:
num_classes = 10
input_shape = (1, 28, 28, 1)
output_shape = (1, num_classes)

In [4]:
client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')
cluster = sy.TFECluster(alice, bob, carol)

client.connect_to_model(input_shape, output_shape, cluster)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## क्वेरी मॉडल

आपण काही खाजगी अंदाज घेण्यासाठी तयार आहात! `query_model` ला कॉल करणे ही `image` वर तयार केलेल्या रांगेत समाविष्ट केली जाईल, स्थानिक पातळीवर डेटा सामायिक करेल आणि **भाग 13 बी** मधील मॉडेल सर्व्हरवर शेअर्स सबमिट करेल.

In [5]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [6]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "wrongly",
        predicted_label))

The image had label 7 and was correctly classified as 7
The image had label 2 and was correctly classified as 2
The image had label 1 and was correctly classified as 1


हे उत्तम झाले. आपण या तीन प्रतिमांचे योग्य प्रकारे वर्गीकरण करण्यास सक्षम आहात! परंतु या भविष्यवाणींमध्ये विशेष म्हणजे आपण ही सेवा मिळविण्यासाठी कोणतीही खाजगी माहिती उघड केली नाही. मॉडेल होस्टने आपला इनपुट डेटा किंवा आपल्या अंदाज कधीही पाहिले नाहीत आणि आपण मॉडेल कधीही डाउनलोड केले नाही. आपण कूटबद्ध केलेल्या मॉडेलसह कूटबद्ध केलेल्या डेटावर खाजगी अंदाज प्राप्त करण्यास सक्षम होता!

आमच्या स्वतःच्या अ‍ॅप्समध्ये हे लागू करण्यासाठी आपण गर्दी करण्यापूर्वी, आपल्या सर्व्ह केलेले मॉडेल साफ करण्यासाठी त्वरीत **भाग 13 बी** वर परत जाऊया!